In [ ]:
!pip install pycocotools

In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import os
from collections import defaultdict, Counter
import numpy as np

In [ ]:
# Pfade zu den Annotationendateien (Ersetzen mit eigenen Pfaden)
train_annotation_file = r"   " # Pfad zu Trainingsannotations
val_annotation_file = r"   " # Pfad zu Validierungsannotations
test_annotation_file = r"   " # Pfad zu Testannotations

In [ ]:
# Laden der Annotationen
coco_train = COCO(train_annotation_file) # Trainingssatz laden
coco_val = COCO(val_annotation_file) # Validierungssatz laden
coco_test = COCO(test_annotation_file) # Testsatz laden

In [ ]:
# Anzahl der Annotationen und Bilder in jedem Datensatz
# Trainingssatz
train_annotation_ids = coco_train.getAnnIds()
num_train_annotations = len(train_annotation_ids)
num_train_images = len(coco_train.getImgIds())
print(f"Anzahl der Annotationen im Trainingssatz: {num_train_annotations}")
print(f"Anzahl der Bilder im Trainingssatz: {num_train_images}")

# Validierungssatz
val_annotation_ids = coco_val.getAnnIds()
num_val_annotations = len(val_annotation_ids)
num_val_images = len(coco_val.getImgIds())
print(f"Anzahl der Annotationen im Validierungssatz: {num_val_annotations}")
print(f"Anzahl der Bilder im Validierungssatz: {num_val_images}")

# Testsatz
test_annotation_ids = coco_test.getAnnIds()
num_test_annotations = len(test_annotation_ids)
num_test_images = len(coco_test.getImgIds())
print(f"Anzahl der Annotationen im Testsatz: {num_test_annotations}")
print(f"Anzahl der Bilder im Testsatz: {num_test_images}")

# Gesamtanzahl der Annotationen und Bilder
total_annotations = num_train_annotations + num_val_annotations + num_test_annotations
total_images = num_train_images + num_val_images + num_test_images
print(f"Gesamtanzahl der Annotationen: {total_annotations}")
print(f"Gesamtanzahl der Bilder: {total_images}")

In [ ]:
# Durchschnittliche Anzahl von Annotationen pro Bild
# Trainingssatz
durchschn_train = num_train_annotations / num_train_images
print(f"Durchschnittliche Annotationen pro Bild im Trainingssatz: {durchschn_train:.2f}")

# Validierungssatz
durchschn_val = num_val_annotations / num_val_images
print(f"Durchschnittliche Annotationen pro Bild im Validierungssatz: {durchschn_val:.2f}")

# Testsatz
durchschn_test = num_test_annotations / num_test_images
print(f"Durchschnittliche Annotationen pro Bild im Testsatz: {durchschn_test:.2f}")

# Gesamt
durchschn_total = total_annotations / total_images
print(f"Durchschnittliche Annotationen pro Bild insgesamt: {durchschn_total:.2f}")

In [ ]:
# Verteilung der Bounding Box Breiten und Höhen im Trainingssatz
bbox_widths, bbox_heights = [], []
for ann in coco_train.dataset['annotations']:
    width, height = ann['bbox'][2], ann['bbox'][3] # Breite und Höhe der Bounding Box
    bbox_widths.append(width)
    bbox_heights.append(height)

In [ ]:
# Histogramme erstellen
plt.figure(figsize=(10, 6))
plt.hist(bbox_widths, bins=50)
plt.xlabel('Breite der Bounding Box')
plt.ylabel('Anzahl')
plt.title('Verteilung der Bounding Box Breiten im Trainingssatz')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(bbox_heights, bins=50)
plt.xlabel('Höhe der Bounding Box')
plt.ylabel('Anzahl')
plt.title('Verteilung der Bounding Box Höhen im Trainingssatz')
plt.show()

In [ ]:
# Anzahl verschiedener Bildauflösungen im Trainingssatz
image_resolutions = [(img['width'], img['height']) for img in coco_train.dataset['images']]
unique_resolutions = set(image_resolutions)
print(f"Anzahl verschiedener Bildauflösungen im Trainingssatz: {len(unique_resolutions)}")
print("Verschiedene Bildauflösungen im Trainingssatz:")
for res in unique_resolutions:
    print(f"Breite: {res[0]}, Höhe: {res[1]}")

In [ ]:
# Verteilung der Anzahl der Annotationen pro Bild
def compute_annotation_distribution(coco, dataset_name):
    image_annotation_count = defaultdict(int)
    for ann in coco.dataset['annotations']:
        image_annotation_count[ann['image_id']] += 1

    images_with_zero_annotations = sum(1 for image_id in coco.getImgIds() if image_annotation_count.get(image_id, 0) == 0)
    images_with_one_annotation = sum(1 for count in image_annotation_count.values() if count == 1)
    images_with_two_to_five_annotations = sum(1 for count in image_annotation_count.values() if 2 <= count <= 5)
    images_with_more_than_five_annotations = sum(1 for count in image_annotation_count.values() if count > 5)

    print(f"\nStatistik für den {dataset_name}-Datensatz:")
    print(f" - Bilder ohne Annotation: {images_with_zero_annotations}")
    print(f" - Bilder mit einer Annotation: {images_with_one_annotation}")
    print(f" - Bilder mit 2 bis 5 Annotationen: {images_with_two_to_five_annotations}")
    print(f" - Bilder mit mehr als 5 Annotationen: {images_with_more_than_five_annotations}")

In [ ]:
# Trainings-, Validierungs- und Testsatz analysieren
compute_annotation_distribution(coco_train, "Trainings")
compute_annotation_distribution(coco_val, "Validierungs")
compute_annotation_distribution(coco_test, "Test")

In [ ]:
# Gruppierte Verteilung der Anzahl der Annotationen pro Bild plotten
def plot_annotation_count_distribution_grouped(coco, dataset_name):
    annotation_counts = [len(coco.getAnnIds(imgIds=[image_id])) for image_id in coco.getImgIds() if len(coco.getAnnIds(imgIds=[image_id])) > 0]
    grouped_counts = defaultdict(int)
    for count in annotation_counts:
        if count == 1:
            grouped_counts['1'] += 1
        elif count == 2:
            grouped_counts['2'] += 1
        elif count == 3:
            grouped_counts['3'] += 1
        elif 4 <= count <= 5:
            grouped_counts['4-5'] += 1
        elif 6 <= count <= 10:
            grouped_counts['6-10'] += 1
        else:
            grouped_counts['>10'] += 1

    group_order = ['1', '2', '3', '4-5', '6-10', '>10']
    image_frequencies = [grouped_counts.get(group, 0) for group in group_order]

    plt.figure(figsize=(8, 6))
    plt.bar(group_order, image_frequencies)
    plt.xlabel('Anzahl der Annotationen pro Bild')
    plt.ylabel('Anzahl der Bilder')
    plt.title(f'Verteilung der Anzahl der Annotationen pro Bild im {dataset_name}-Datensatz (Gruppiert)')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
# Verteilungen plotten
plot_annotation_count_distribution_grouped(coco_train, "Trainings")
plot_annotation_count_distribution_grouped(coco_val, "Validierungs")
plot_annotation_count_distribution_grouped(coco_test, "Test")